<a href="https://colab.research.google.com/github/gokceuludogan/protein-ml-crash-course/blob/wip/Chapter_4_Secondary_Structure_Prediction_with_LSTM_model_%26_Pytorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

In this chapter, we will predict **protein secondary structure**, which refers to the local folding of the protein's backbone into common motifs such as **alpha helices**, **beta sheets**, and **random coils**. This is a classic problem in bioinformatics that can be addressed using machine learning (ML) or deep learning (DL) techniques.

We will:

- Understand the protein secondary structure prediction task.
- Encode protein sequences and their corresponding secondary structure labels.
- Use DL techniques, specifically Recurrent Neural Networks, to build predictive models.
- Evaluate the performance of these models.

---

## 1. Protein Secondary Structure

### Definitions:

- **Primary structure**: The linear sequence of amino acids in the protein.
- **Secondary structure**: Local folding patterns of the protein chain, mainly forming alpha helices (H), beta sheets (E), or random coils (C).
- **Tertiary structure**: The overall 3D structure of the protein.

### Common Classes in Secondary Structure Prediction:

- **H**: Alpha-helix
- **E**: Beta-sheet
- **C**: Coil (random structure)

### Dataset Example:

We will work with a dataset where each protein sequence is accompanied by its secondary structure labels.

| Sequence | Secondary Structure |
| --- | --- |
| MAGWELV | HCCCCCC |
| GGQVNLL | CCECCEE |

---

## 2. Data Preprocessing

For this task, we will use the UniProt SwissProt database, which contains manually curated protein sequences. We will download the sequences in FASTA format and compute their secondary structures using DSSP (Define Secondary Structure of Proteins), a tool that assigns secondary structure elements (alpha-helix, beta-sheet, coil) based on 3D coordinates of the protein's structure.

### Download UniProt SwissProt Database

We will download the SwissProt database from the UniProt FTP server. This database contains high-quality, manually reviewed protein sequences.




In [ ]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
!gunzip uniprot_sprot.fasta.gz

--2024-10-04 16:05:50--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92678508 (88M) [application/x-gzip]
Saving to: ‘uniprot_sprot.fasta.gz.1’

uniprot_sprot.fasta  23%[===>                ]  20,62M  3,85MB/s    eta 20s    

In [ ]:
!pip install biopython

In [ ]:
import random
from Bio import SeqIO

# Set the file path for SwissProt
swissprot_fasta = "uniprot_sprot.fasta"

# Load SwissProt sequences and randomly sample 1000
sequences = list(SeqIO.parse(swissprot_fasta, "fasta"))

# Sample 1000 sequences
sampled_sequences = random.sample(sequences, 100)

# Extract UniProt IDs (they are in the FASTA header)
uniprot_ids = [seq.id.split('|')[1] for seq in sampled_sequences]


### Download AlphaFold structures using Bio.PDB.alphafold_db


In [ ]:
import requests
import os
# Directory to store AlphaFold structures
output_dir = "alphafold_structures"
os.makedirs(output_dir, exist_ok=True)

# Function to download AlphaFold structure from API
def download_alphafold_structure(uniprot_id, output_dir):
    url = f"https://alphafold.com/api/prediction/{uniprot_id}"
    response = requests.get(url)
    if response.status_code == 200:
        response = requests.get(response.json()[0]['pdbUrl'])
        if response.status_code != 200:
            print(f"Failed to download structure for {uniprot_id}: {response.status_code}")
            return None
        output_file = os.path.join(output_dir, f"{uniprot_id}.pdb")
        with open(output_file, "w") as f:
            f.write(response.text)
        print(f"Downloaded AlphaFold structure for {uniprot_id}")
        return output_file
    else:
        print(f"Failed to download structure for {uniprot_id}: {response.status_code}")
        return None

# Download AlphaFold structures for sampled UniProt IDs
pdb_files = []
for uniprot_id in uniprot_ids:
    pdb_file = download_alphafold_structure(uniprot_id, output_dir)
    if pdb_file:
        pdb_files.append(pdb_file)


/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Downloaded AlphaFold structure for Q4L6X8
Downloaded AlphaFold structure for Q8E3Y3
Downloaded AlphaFold structure for Q66I71
Downloaded AlphaFold structure for Q6C6N0
Downloaded AlphaFold structure for Q6NDE9
Downloaded AlphaFold structure for Q9ASR3
Downloaded AlphaFold structure for Q2Y7P0
Downloaded AlphaFold structure for Q3J446
Downloaded AlphaFold structure for Q3Z8Z7
Downloaded AlphaFold structure for P36438
Downloaded AlphaFold structure for B0U0T8
Downloaded AlphaFold structure for A4R962
Downloaded AlphaFold structure for Q8N1N0
Failed to download structure for Q06658: 404
Downloaded AlphaFold structure for P85115
Downloaded AlphaFold structure for B3ELD9
Downloaded AlphaFold structure for A0RMU4
Downloaded AlphaFold structure for Q9FN93
Downloaded AlphaFold structure for P57722
Downloaded AlphaFold structure for B2S046
Downloaded AlphaFold structure for Q65CR5
Downloaded AlphaFold structure for P05342
Downloaded AlphaFold structure for Q7NNK8
Downloaded AlphaFold structure 

In [ ]:
!apt-get install dssp

zsh:1: command not found: apt-get


### Compute Secondary Structures using DSSP.

In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB import DSSP

parser = PDBParser(QUIET=True)

# Dictionary to store UniProt ID and secondary structure assignment
secondary_structures = {}

for pdb_file in pdb_files:
    try:
        # Parse the AlphaFold structure
        structure = parser.get_structure("model", pdb_file)
        model = structure[0]

        # Apply DSSP to the AlphaFold model
        dssp = DSSP(model, pdb_file)

        # Extract secondary structure assignment
        sec_struct = [item[2] for item in dssp]
        uniprot_id = os.path.basename(pdb_file).split('.')[0]
        secondary_structures[uniprot_id] = sec_struct

    except Exception as e:
        print('Error in computing secondary structure for ', pdb_file)
# Save secondary structures to file
import json
with open("secondary_structures.json", "w") as f:
    json.dump(secondary_structures, f)

print("Secondary structure computation completed and saved.")

/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/Bio/PDB/DSSP.py:199: UserWarning: Trying to parse 'DAUC'
Error parsing PDB at line 38
Not a valid integer in PDB record

  warnings.warn(err)


Error in computing secondary structure for  alphafold_structures/A0A161X1M2.pdb
Secondary structure computation completed and saved.


### Encoding Sequences and Structures

Similar to Chapter 3, we will **one-hot encode** the amino acid sequences. The secondary structure labels will also be encoded, but instead of one-hot encoding, we can map each label to a unique integer.

In [ ]:
import numpy as np
from Bio import SeqIO

# Define amino acids and secondary structure mappings
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}
structure_to_int = {'H': 0, 'E': 1, 'T': 2, 'B': 3, 'S': 4, 'G': 5, 'P': 6, 'I': 7, '-': 8}

# Function to read sequences from a FASTA file
def read_fasta_to_dict(fasta_file):
    seq_dict = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_dict[record.id.split('|')[1]] = str(record.seq)
    return seq_dict

# Function to encode sequences
def one_hot_encode_sequence(sequence):
    one_hot = np.zeros((len(sequence), len(amino_acids)))
    for i, aa in enumerate(sequence):
        if aa in aa_to_int:
            one_hot[i, aa_to_int[aa]] = 1
    return one_hot

# Function to encode secondary structure
def encode_structure(structure):
    one_hot = np.zeros((len(structure), len(structure_to_int)))
    for i, ss in enumerate(structure):
        if ss in structure_to_int:
            one_hot[i, structure_to_int[ss]] = 1
    return one_hot


# Function to pad or truncate sequences
def pad_or_truncate_sequence(sequence, target_length):
    if len(sequence) > target_length:
        return sequence[:target_length]
    elif len(sequence) < target_length:
        padding_length = target_length - len(sequence)
        return np.pad(sequence, ((0, padding_length), (0, 0)), 'constant')
    else:
        return sequence


# Example usage
fasta_file = "uniprot_sprot.fasta"  # Path to your FASTA file
sequence_dict = read_fasta_to_dict(fasta_file)

# Initialize lists to store the processed data
X_windows = []
encoded_structure = []

# Define a fixed window size
window_size = 100  # You can choose the appropriate size based on your data

for uniprot_id, ss in secondary_structures.items():

    if uniprot_id in sequence_dict:
        sequence = sequence_dict[uniprot_id]

        # One-hot encode the amino acid sequence
        encoded_sequence = one_hot_encode_sequence(sequence)

        # Pad or truncate to ensure fixed length
        padded_sequence = pad_or_truncate_sequence(encoded_sequence, window_size)

        X_windows.append(padded_sequence)
        encoded_structure.append(pad_or_truncate_sequence(encode_structure(ss), window_size))



# Convert to numpy arrays
X_windows = np.array(X_windows)  # Shape: (num_samples, window_size, num_features)
encoded_structure = np.array(encoded_structure)  # Shape: (num_samples, window_size)


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_windows, encoded_structure, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create DataLoaders for training and validation sets
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

## Deep Learning for Secondary Structure Prediction

Deep learning models, such as **Recurrent Neural Networks (RNNs)** or **Convolutional Neural Networks (CNNs)**, can capture sequential patterns more effectively for this type of task.

### LSTM (Long Short-Term Memory) Model

LSTMs are a type of RNN that can capture long-range dependencies, making them a powerful choice for sequence-based tasks like secondary structure prediction.

In this notebook, we will implement a deep learning model using PyTorch Lightning, a framework that simplifies the training and evaluation process for PyTorch models. PyTorch Lightning helps manage the boilerplate code, making it easier to scale models and manage experiments.

### PyTorch Lightning Setup
The LSTM model will be wrapped inside a PyTorch Lightning module, which handles the forward pass, training, and validation steps for us. This module will allow us to train, validate, and predict efficiently while maintaining flexibility for future modifications.

In [ ]:
!pip install pytorch-lightning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from torchmetrics import Accuracy

# Define the PyTorch Lightning module
class LSTMModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, learning_rate=1e-3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=output_size)
        self.learning_rate = learning_rate

    # Forward pass
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)  # Apply fully connected layer to each time step
        return out # Shape: (batch_size, window_size, output_size)

    # Training step (one batch)
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        # Reshape outputs and labels for loss calculation (flatten time steps and batch size)
        outputs = outputs.view(-1, outputs.shape[-1])  # (batch_size * window_size, output_size)
        labels = labels.view(-1, labels.shape[-1])  # (batch_size * window_size)
        loss = self.loss_fn(outputs, labels)
        acc = self.accuracy(torch.argmax(outputs, dim=1), torch.argmax(labels, dim=1))
        self.log('train_loss', loss)
        self.log('train_acc', acc, prog_bar=True)
        return loss


    # Validation step (one batch)
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1, labels.shape[-1])
        loss = self.loss_fn(outputs, labels)
        acc = self.accuracy(torch.argmax(outputs, dim=1), torch.argmax(labels, dim=1))
        self.log('val_loss', loss)
        self.log('val_acc', acc, prog_bar=True)

    # Predict method to return predicted labels
    def predict(self, x):
        self.eval()  # Set model to evaluation mode
        with torch.no_grad():
            outputs = self(x)  # Forward pass
            predicted_labels = torch.argmax(outputs, dim=2)  # Get predicted class (max value along class dimension)
        return predicted_labels

    # Optimizer setup
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)



# Define the model
input_size = X_windows.shape[2]  # Number of features (e.g., amino acids)
hidden_size = 64  # Number of LSTM units
output_size = 9   # Number of classes

# Create an instance of the model
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

# Initialize the PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=10, accelerator="auto", devices="auto")  # Automatically uses GPU if available

# Train the model using the Trainer (like model.fit)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

# Optionally, evaluate the model on a test set (similar to model.evaluate)
trainer.validate(model, dataloaders=val_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | lstm     | LSTM               | 22.0 K | train
1 | fc       | Linear             | 585    | train
2 | loss_fn  | CrossEntropyLoss   | 0      | train
3 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
22.6 K    Trainable params
0         Non-trainable params
22.6 K    Total params
0.090     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you wa

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc             0.277368426322937
        val_loss            1.8194397687911987
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 1.8194397687911987, 'val_acc': 0.277368426322937}]

## 5. Model Evaluation

We can evaluate the performance of our models using metrics such as **accuracy**, **precision**, and **recall**. In secondary structure prediction, it's important to measure the performance for each class individually as well as the overall performance.

In [ ]:
from sklearn.metrics import classification_report


# Get predictions for the validation data
predictions = model.predict(X_val_tensor)

# LSTM Evaluation (for demonstration, evaluating on training data)
lstm_pred = model.predict(X_val_tensor)

# Flatten both predictions and true labels to compare them element-wise
lstm_pred_flat = lstm_pred.view(-1).cpu().numpy()  # Flatten the predicted labels and move to CPU
y_val_flat = torch.argmax(y_val_tensor, dim = 2) # Flatten the true labels and move to CPU
y_val_flat = y_val_flat.view(-1).cpu().numpy()
# Generate the classification report
target_names = ['Helix (H)', 'Sheet (E)', 'Coil (C)', 'Turn (T)', 'Bend (B)', 'Bridge (G)', 'Polyproline (P)', 'I-helix (I)', 'Other (-)']
lstm_report = classification_report(y_val_flat, lstm_pred_flat, target_names=target_names)

# Print the classification report
print("LSTM Classification Report:\n", lstm_report)

LSTM Classification Report:
                  precision    recall  f1-score   support

      Helix (H)       0.36      0.86      0.51       528
      Sheet (E)       0.40      0.34      0.37       381
       Coil (C)       0.30      0.02      0.03       185
       Turn (T)       0.00      0.00      0.00        13
       Bend (B)       0.00      0.00      0.00       143
     Bridge (G)       0.00      0.00      0.00        37
Polyproline (P)       0.00      0.00      0.00        94
    I-helix (I)       0.00      0.00      0.00        10
      Other (-)       0.48      0.31      0.38       509

       accuracy                           0.39      1900
      macro avg       0.17      0.17      0.14      1900
   weighted avg       0.34      0.39      0.32      1900



/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ozdeniz/Desktop/crashcourse/protein-ml-crash-course/ccourse/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

## 6. Conclusion

In this chapter, we tackled the problem of secondary structure prediction and demonstrated how to create a dataset from UniProt SwissProt. We then explored advanced deep learning architectures, specifically LSTMs (Long Short-Term Memory networks), to predict protein secondary structures based on amino acid sequences. Additionally, we introduced the use of PyTorch Lightning to streamline the training and evaluation process, making our deep learning workflow more efficient and scalable.